In [1]:
import pandas as pd
import numpy as np

from pyomo.environ import *

from data_cleaning import hub_occ_dict
from data_cleaning import heatdays_df
from data_cleaning import cengeo_pop_dict
from data_cleaning import bg_ces_df

dist_to_hub_df = pd.read_csv('data/distmatrix_contracosta.csv')
dist_to_hub_df.set_index('Unnamed: 0', inplace = True)
dist_to_hub_df.index.name = None

### Define Model

_Create model and variable indices_

In [2]:
model = ConcreteModel()

model.idx_cengeos = Set(initialize = dist_to_hub_df.index)
model.idx_hubs = Set(initialize = dist_to_hub_df.columns)

_Create set of blockgroup, hub pairs within five miles of each other_

In [3]:
def filter_to_nearby_hubs(model, cg, hub):
    return not np.isnan(dist_to_hub_df.loc[cg, hub])

model.idx_cg_hub_nearby_pairs = Set(initialize = model.idx_cengeos*model.idx_hubs, filter = filter_to_nearby_hubs)

### Define Parameters

_Occupancy parameter indexed on hub_

In [4]:
def get_hub_occ(model, hub):
    return(hub_occ_dict[hub])

model.param_hub_occ = Param(model.idx_hubs, initialize = get_hub_occ)

_Pairwise distance indexed on (blockgroup, hub) tuple_

In [5]:
def get_cg_hub_distance(model, cg, hub):
    return dist_to_hub_df.loc[cg, hub]

model.cg_hub_distance = Param(model.idx_cg_hub_nearby_pairs, initialize = get_cg_hub_distance)

### Define Variables

_Is this site a hub?_

In [6]:
model.var_hub_yn = Var(model.idx_hubs, initialize = 0, within = Binary)

_What proportion of a blockgroup's population is served by this hub?_

In [7]:
model.var_prop_served = Var(model.idx_cg_hub_nearby_pairs, initialize = 0.0, bounds = (0.0, 1.0))

_How many people are served by this hub?_

In [8]:
def ppl_served_at_hub(model, hub):
    tot_ppl_served = 0

    for cg in model.idx_cengeos:
        if (cg, hub) in model.idx_cg_hub_nearby_pairs:
            ppl_served = model.var_prop_served[cg, hub]*cengeo_pop_dict[cg]
            tot_ppl_served = ppl_served + tot_ppl_served
        else:
            tot_ppl_served = tot_ppl_served
    
    return(tot_ppl_served)

model.var_ppl_at_hub = Var(model.idx_hubs, initialize = ppl_served_at_hub, domain = NonNegativeReals)

_How many people are served in this blockgroup?_

In [9]:
def ppl_served_in_cengeo(model, cg):
    tot_ppl_served = 0

    for hub in model.idx_hubs:
        if (cg, hub) in model.idx_cg_hub_nearby_pairs:
            ppl_served = model.var_prop_served[cg, hub]*cengeo_pop_dict[cg]
            tot_ppl_served = ppl_served + tot_ppl_served
        else:
            tot_ppl_served = tot_ppl_served
    
    return(tot_ppl_served)

model.var_served_in_cg = Var(model.idx_cengeos, initialize = ppl_served_in_cengeo, domain = NonNegativeReals)

### Define Objective

_Minimize aggregate travel distance_

In [10]:
min_agg_dist = sum(model.cg_hub_distance[cg, hub] * cengeo_pop_dict[cg] * model.var_prop_served[cg, hub] 
                   for cg, hub in model.idx_cg_hub_nearby_pairs)

model.obj_min_agg_dist = Objective(expr = min_agg_dist, sense = minimize)

### Define Constraints

_Hub cannot serve more people than it has capacity for_

In [11]:
def serve_less_than_occ(model, hub):
    return(0.9*hub_occ_dict[hub], model.var_ppl_at_hub[hub], hub_occ_dict[hub])

model.con_max_occ = Constraint(model.idx_hubs, rule = serve_less_than_occ)

_Construct a set number of hubs_

In [12]:
model.con_max_hubs = Constraint(expr = sum(model.var_hub_yn[hub] for hub in model.idx_hubs) == 3)

_No one is served at a location if it is not deemed a hub_

In [13]:
def open_only(model, cg, hub):
    return(model.var_prop_served[cg, hub] <= model.var_hub_yn[hub])

model.con_open_only = Constraint(model.idx_cg_hub_nearby_pairs, rule = open_only)

_Meet a certain portion of demand in area of interest_

In [14]:
sum_ppl_served = sum(model.var_ppl_at_hub[hub] for hub in model.idx_hubs) 
sum_area_pop = sum(cengeo_pop_dict[cg] for cg in model.idx_cengeos)

model.con_min_coverage = Constraint(expr = sum_ppl_served >= 0.5*sum_area_pop)

_Prioritize CalEnviroScreen populations_

In [15]:
model.con_serve_ces = ConstraintList()

for cg in model.idx_cengeos:
    cg_ces_score = float(bg_ces_df.loc[bg_ces_df['GISJOIN'] == cg, 'SCORE_PCTL_CI_BG'])

    if cg_ces_score >= 75.0:
        model.con_serve_ces.add(expr = model.var_served_in_cg[cg] >= 0.95*cengeo_pop_dict[cg])

### Solve Model

In [ ]:
# Run Model
from pyomo.opt import SolverFactory

SolverFactory('glpk').solve(model)

var_popu_served = [model.var_popu_served[hub].value for hub in model.hubs]
var_hub_yn = [model.var_hub_yn[hub].value for hub in model.hubs]
prop_served_list = []

for cg in model.cengeos:
    cg_dict = dict()
    for pair in model.cg_hub_nearby_pairs:
        if pair[0]==cg:
            cg_dict[pair[1]] = model.var_prop_served[pair].value
    prop_served_list.append(cg_dict)

var_prop_served = pd.DataFrame(prop_served_list)